In [285]:
import numpy as np

In [288]:
class NeuralNetwork:
    def __init__(self, input_size=718, hidden_layers=[512, 512], output_size=3):
        self.input_size = input_size
        self.hidden_layers = hidden_layers
        self.output_size = output_size
        self.weights = []
        self.biases = []

        # Input to Hidden Layers Network
        self.weights.append(0.01 * np.random.rand(input_size, hidden_layers[0]))
        self.biases.append(np.zeros((1, hidden_layers[0])))

        # Hidden Layers Network
        for i in range(len(hidden_layers)-1):
            self.weights.append(0.01 * np.random.rand(hidden_layers[i], hidden_layers[i+1]))
            self.biases.append(np.zeros((1, hidden_layers[i+1])))
        
        # Hidden Layers Network to Output
        self.weights.append(0.01 * np.random.rand(hidden_layers[len(hidden_layers)-1], output_size))
        self.biases.append(np.zeros((1, output_size)))

    def forward(self, inputs):
        layers = [inputs]

        for i in range(len(self.weights)):
            # Dot Product to 
            layers.append(np.dot(layers[-1], self.weights[i]) + self.biases[i])

            # Activation Functions (ReLU + SoftMax)
            if i == len(self.weights)-1:
                finalOutput = np.exp(layers[-1] - np.max(layers[-1], axis=1, keepdims=True))
                finalOutput = finalOutput / np.sum(finalOutput, axis=1, keepdims=True)
                layers.append(finalOutput)
            else:
                layers.append(np.maximum(0, layers[-1]))
        
        return layers[-1]


In [289]:
# LossCategoricalCrossEntropy implementation
def LossCategoricalCrossEntropy(yPred, yTrue):
    # If predicted class has a prediction of 0% likelihood this prevents log(0), which would be infinity
    yPred = np.clip(yPred, 1e-10, 1 - 1e-10)

    # We calculate the sum of the log losses
    loss = -np.sum(yTrue * np.log(yPred), axis=1)

    # We calculate the average loss - this depends on the number of samples
    # So the return loss is the average loss not the summed up loss (which took me a while to understand)
    average_loss = np.mean(loss)

    return average_loss


In [290]:
myNeuralNet = NeuralNetwork()

In [292]:
result = myNeuralNet.forward(np.random.rand(1, 718))
result

array([[0.34324246, 0.34636421, 0.31039333]])

# Notes

(batchNumber[32], inputs[718]) * (weights[718], weights[512]) = (32, 512)

- Implement GitHub Repo - DONE

Forward pass - Train 
    - Neurons + - DONE
    - Acitvation function + - DONE
    - Softwax (get probablities) + - DONE
    - loss
    - Test Forward Pass with Other Model Gradients

Implement backpropagation
    - Explain how to do and everything
    - Start training with basic 80-20

Introduce regularization
Batch Normalization

Optimizer

Visualize gradients

Visualize MNIST with UMAP or T-SNE